In [151]:
# Import dependencies 
import numpy as np
from twitter import *
import pandas as pd 
import datetime

In [152]:
#The tokens and consumers keys are my API authentication to use the Twitter API.  I'm using the twitter library as a python
#wrapper to call twitter request through python.
token = '1212463191910842368-l00ryMICeXQXXawl8w2zazEoFS7xDf'
token_secret = 'hbzjFRq1r2ygkydPXfrwrhm5zTa54F3mYEm2wi7Q09DgA'
consumer_key = 'g8V7jnF3dqMfg4LRhtobYB4Pl'
consumer_secret = 'Y0SHSL0H9X9gCtuy07mJ3cp144DS2JhwX4Uvgda2ph8NvIUswJ'

t = Twitter(
        auth=OAuth(token, token_secret, consumer_key, consumer_secret))

In [153]:
type(t)

twitter.api.Twitter

In [154]:
# The function below gets a person's timeline
# Note: Count can't be greater than 200
#x = t.statuses.user_timeline(screen_name="BernieSanders", count=20, include_rts=False, tweet_mode = 'extended')

In [155]:
# # Creating a for loop to inserts Hillary's tweets in a list
# Bernie_tweet = []
# for x in x:
#     Bernie_tweet.append(x)

In [156]:
# Bernie_tweet[0:5]

In [159]:
# type(Bernie_tweet)

In [160]:
# # I just want to focus on ids and tweets
# for element in Bernie_tweet:
#     print (element['id'])
#     print (element['full_text'])
#     print('---')

In [161]:
# #(TweetMiner) I found this function that easily organizes the tweets to input them into a panda dataframe

# class TweetMiner(object):

    
#     def __init__(self, t, result_limit = 200):
        
#         self.t = t        
#         self.result_limit = result_limit
        

#     def mine_user_tweets(self, user="BernieSanders", mine_retweets=False, max_pages=20):

#         data           =  []
#         last_tweet_id  =  False
#         page           =  1
        
#         while page <= max_pages:
            
#             if last_tweet_id:
#                 statuses   =   self.t.statuses.user_timeline(screen_name=user, count=self.result_limit, max_id=last_tweet_id - 1, include_rts=mine_retweets, tweet_mode = 'extended')
#                 statuses = [x for x in statuses]
#             else:
#                 statuses   =   self.t.statuses.user_timeline(screen_name=user, count=self.result_limit, include_rts=mine_retweets, tweet_mode = 'extended')
#                 statuses = [x for x in statuses]
                
#             for item in statuses:
#                 # Using try except here.
#                 # When retweets = 0 we get an error (GetUserTimeline fails to create a key, 'retweet_count')
#                 try:
#                     mined = {
#                         'tweet_id':        item['id'],
#                         'handle':          item['user']['screen_name'],
#                         'retweet_count':   item['retweet_count'],
#                         'text':            item['full_text'],
#                         'mined_at':        datetime.datetime.now(),
#                         'created_at':      item['created_at'],
#                     }
                
#                 except:
#                         mined = {
#                         'tweet_id':        item['id'],
#                         'handle':          item['user']['screen_name'],
#                         'retweet_count':   0,
#                         'text':            item['full_text'],
#                         'mined_at':        datetime.datetime.now(),
#                         'created_at':      item['created_at'],
#                     }
                
#                 last_tweet_id = item['id']
#                 data.append(mined)
                
#             page += 1
            
#         return data

In [162]:
# # Result limit == count parameter from our GetUserTimeline()
# miner = TweetMiner(t, result_limit=200)

In [163]:
# bernie = miner.mine_user_tweets(user="BernieSanders")
# elizabeth = miner.mine_user_tweets(user="ewarren")

In [164]:
# # each tweet is dictionary within a list
# bernie[0:5]

In [165]:
# for x in range(5):
#     print (bernie[x]['text'])
#     print('---')

In [166]:
# for x in range(5):
#     print (elizabeth[x]['text'])
#     print('--')

In [167]:
# # Turning Bernie's tweet into a dataframe:
# bernie_df = pd.DataFrame(bernie)
# bernie_df.head()

In [168]:
# # Doing the same for Elizabeth Warren
# miner = TweetMiner(t, result_limit=200)
# elizabeth_tweets = miner.mine_user_tweets("ewarren", max_pages=30)

In [169]:
# elizabeth_df = pd.DataFrame(elizabeth_tweets)

In [170]:
# elizabeth_df.head()

In [171]:
# bernie_df.shape

In [172]:
# elizabeth_df.shape

In [173]:
# # Joined elizabeth & Bernie Dataframe
# tweets = pd.concat([elizabeth_df, bernie_df], axis=0)
# tweets.shape

In [174]:
# tweets.head()

In [175]:
# The library below convert a collection of text documents to a matrix of token counts
from sklearn.feature_extraction.text import TfidfVectorizer

# The library stores dictionary values as counts 
from collections import Counter

In [176]:
# # Using TfidfVectorizer to find ngrams for Elizabeth's tweets
# vect = TfidfVectorizer(ngram_range=(2,5), stop_words='english')
# vect

In [177]:
# # Pulls all of Elizabeth's tweet text's into one giant string
# summaries = "".join(elizabeth_df['text'])
# summaries

In [178]:
# # vect.builder_analyzer handles the preprocessing, tokenization and n-grams generation
# ngrams_summaries = vect.build_analyzer()(summaries)
# ngrams_summaries

In [179]:
# Counter(ngrams_summaries).most_common(20)

In [180]:
# #Doing the same process for Bernie
# vect = TfidfVectorizer(ngram_range=(2,5), stop_words='english')

# summaries = "".join(bernie_df['text'])
# ngrams_summaries = vect.build_analyzer()(summaries)

# Counter(ngrams_summaries).most_common(20)

In [181]:
# #Wine search to look for #wine in the full text.  Twee_mode='extended' reflects the full tweet, not truncated.
# wine = t.search.tweets(q='$aapl', include_rts=False, tweet_mode='extended')

In [182]:
# type(wine)

In [183]:
# wine

In [184]:
# wine_list = wine['statuses']

In [185]:
# type(wine_list)

In [186]:
# wine_list

In [187]:
# # Showing how #wine reflects within the full text
# for element in wine_list[0:5]:
#     print (element['id'])
#     print (element['full_text'])
#     print('---')

In [188]:
# #(TweetMiner) I found this function that easily organizes the tweets to input them into a panda dataframe

# class TweetMiner(object):

    
#     def __init__(self, t, result_limit = 200):
        
#         self.t = t        
#         self.result_limit = result_limit
        

#     def mine_user_tweets(self, topic="#wine", mine_retweets=False, max_pages=20):

#         data           =  []
#         last_tweet_id  =  False
#         page           =  1
        
#         while page <= max_pages:
            
#             if last_tweet_id:
#                 statuses   =   self.t.search.tweets(q='#wine', count=self.result_limit, max_id=last_tweet_id - 1, include_rts=mine_retweets, tweet_mode='extended')
#                 statuses = [x for x in statuses['statuses']]
#             else:
#                 statuses   =   self.t.search.tweets(q='#wine', count=self.result_limit, include_rts=mine_retweets, tweet_mode='extended')
#                 statuses = [x for x in statuses['statuses']]
                
#             for item in statuses:
#                 # Using try except here.
#                 # When retweets = 0 we get an error (GetUserTimeline fails to create a key, 'retweet_count')
#                 try:
#                     mined = {
#                         'tweet_id':        item['id'],
#                         'handle':          item['user']['screen_name'],
#                         'retweet_count':   item['retweet_count'],
#                         'text':            item['full_text'],
#                         'mined_at':        datetime.datetime.now(),
#                         'created_at':      item['created_at'],
#                     }
                
#                 except:
#                         mined = {
#                         'tweet_id':        item['id'],
#                         'handle':          item['user']['screen_name'],
#                         'retweet_count':   0,
#                         'text':            item['full_text'],
#                         'mined_at':        datetime.datetime.now(),
#                         'created_at':      item['created_at'],
#                     }
                
#                 last_tweet_id = item['id']
#                 data.append(mined)
                
#             page += 1
            
#         return data

In [189]:
# miner = TweetMiner(t, result_limit=200)

In [190]:
# wine = miner.mine_user_tweets(topic="$appl")

In [191]:
# wine

In [192]:
# wine_tweet_df = pd.DataFrame(wine)
# wine_tweet_df.head()

In [193]:
# wine_list = (wine_tweet_df.text)

In [194]:
# type(wine_list)

In [195]:
# #Vectorizing the wine df
# vect = TfidfVectorizer(ngram_range=(2,5), stop_words='english')

# summaries = "".join(wine_tweet_df['text'])
# ngrams_summaries = vect.build_analyzer()(summaries)

# Counter(ngrams_summaries).most_common(20)

In [196]:
from textblob import TextBlob

In [197]:
# # Showing tweet sentiment in Bernie df
# # Polarity ranges from -1 (most negative) to 1 (most positive).
# # Subjectivity ranges from 0.0 (very objective) to 1.0 (very subjective)
# for tweet in bernie_df['text']:
#     print(tweet)
#     analysis = TextBlob(tweet)
#     print('---')
#     print(analysis.sentiment)
#     print('---')

In [198]:
# # Showing tweet sentiment in wine
# for tweet in wine_tweet_df['text']:
#     print(tweet)
#     analysis = TextBlob(tweet)
#     print(analysis.sentiment)
#     print('---')

In [199]:
# def detect_polarity(text):
#     return TextBlob(text).sentiment.polarity
#     wine_tweet_df['polarity'] = wine_tweet_df.text.apply(detect_polarity)
# wine_tweet_df.head(100)

In [200]:
ticker_df = pd.read_csv('constituents_csv.csv')

In [201]:
ticker_df.head()

,$-tag,Symbol,Name,Sector
0,$MMM,MMM,3M Company,Industrials
1,$AOS,AOS,A.O. Smith Corp,Industrials
2,$ABT,ABT,Abbott Laboratories,Health Care
3,$ABBV,ABBV,AbbVie Inc.,Health Care
4,$ACN,ACN,Accenture plc,Information Technology


In [202]:
# wine_tweet_df.head()

In [203]:
# wine_tweet_df.shape

In [204]:
aapl_tweet= t.search.tweets(q='$aapl', include_rts=False, tweet_mode='extended', count=100)

In [205]:
type(aapl_tweet)

twitter.api.TwitterDictResponse

In [206]:
aapl_dict = aapl_tweet['statuses']

In [207]:
aapl_dict

[{'created_at': 'Thu Jan 09 03:40:43 +0000 2020',
  'id': 1215116164063539201,
  'id_str': '1215116164063539201',
  'full_text': '1K-1Mill Options 1/8/20 Day 4\nTotal: ($558) Today: ($717.78)\n$TSLA $SPY $AMD $USO $AAPL \nHello, today was full of bad luck I was in BA calls overnight and the plane crash went down and then I had SPY puts which were up huge and then I woke up and they were already back. Head up https://t.co/DpQeRQLonm',
  'truncated': False,
  'display_text_range': [0, 278],
  'entities': {'hashtags': [],
   'symbols': [{'text': 'TSLA', 'indices': [61, 66]},
    {'text': 'SPY', 'indices': [67, 71]},
    {'text': 'AMD', 'indices': [72, 76]},
    {'text': 'USO', 'indices': [77, 81]},
    {'text': 'AAPL', 'indices': [82, 87]}],
   'user_mentions': [],
   'urls': [],
   'media': [{'id': 1215116162255720449,
     'id_str': '1215116162255720449',
     'indices': [279, 302],
     'media_url': 'http://pbs.twimg.com/media/ENz1tiwWkAEPSYF.jpg',
     'media_url_https': 'https://pbs.

In [208]:
tw_list_date = []
tw_list_screen_name = []
tw_list_text = []
tw_list_retweet = []
tw_list_id = []
tw_list_sentiment = []
for element in aapl_dict:
    tw_list_date.append(element['created_at'])
    tw_list_screen_name.append(element['user']['screen_name'])
    tw_list_text.append(element['full_text'])
    tw_list_retweet.append(element['retweet_count'])
    tw_list_id.append(element['id'])
    analysis = TextBlob(element['full_text'])
    tw_list_sentiment.append(analysis.sentiment.polarity)

In [209]:
tw_df = pd.DataFrame(list(zip(tw_list_date, tw_list_screen_name, tw_list_text, tw_list_retweet, tw_list_id, tw_list_sentiment)), 
               columns =['created_date', 'handle', 'text', 'retweet_count', 'tweet_id', 'sentiment'])

In [210]:
tw_df.tail()

,created_date,handle,text,retweet_count,tweet_id,sentiment
95,Thu Jan 09 01:41:59 +0000 2020,stickycactusink,#Korea #information &amp; #telecommunications up 20%\n\nhttps://t.co/HirQ8pxgpR\n\n$KT $WF $SKM $SHG $PKX $KB $LPL $KEP $EWY $KORU $AAPL $AMZN $BA $CMCSA $FB $GOOG $INTC $MSFT $NFLX $NVDA $S $T $TMUS $TSLA $VZ $WMT $XOM #KOSPI #NYSE #KRW #USD https://t.co/wlM5avi8Jq,0,1215086280733839363,0.000000
96,Thu Jan 09 01:41:32 +0000 2020,wallstsolver,🏦 Highest Market Caps 🏦\n🥇Apple Inc.\n$AAPL » 303.19 🔼4.8▪️️1.61%\n🥈Microsoft Corporation\n$MSFT » 160.09 🔼2.51▪️️1.59%\n🥉Alphabet Inc. Class C\n$GOOG » 1404.32 🔼10.98▪️️0.79%\nRankings ➡️ https://t.co/BJATxwgsHz\nhttps://t.co/BJATxwgsHz,0,1215086168364277761,0.000000
97,Thu Jan 09 01:41:27 +0000 2020,tekmunnee,"RT @BlackBoxStocks: A very exciting day in the market for @team_blackbox members, trading our alerts together as a community in one of the…",6,1215086148667822081,0.390000
98,Thu Jan 09 01:39:47 +0000 2020,BlackBoxStocks,"A very exciting day in the market for @team_blackbox members, trading our alerts together as a community in one of the most profitable days yet! $TSLA $EA $AAPL $LK $MSFT 💯📈💰 #WednesdayVibes #Options #OptionsTrading #StockMarket #Stock2020 #Fintech #Financial #stockmarketnews https://t.co/adP9piUBXZ",6,1215085727576469504,0.338333
99,Thu Jan 09 01:36:36 +0000 2020,stickycactusink,$KT Corp. raises #5G #network quality with #AI... AIKON' #technology applied\n\nhttps://t.co/DaXOXKAAoR\n\n$AAPL $AMZN $BA $CMCSA $FB $GOOG $INTC $MSFT $NFLX $NVDA $S $T $TMUS $TSLA $VZ $WMT $XOM #stocks #valueinvesting #money #finance #investors #investing #KOSPI #NYSE #KRW #USD https://t.co/HpGT4RRaTG,0,1215084926451830791,0.000000


In [211]:
#Vectorizing the wine df
vect = TfidfVectorizer(ngram_range=(2,5), stop_words='english')

summaries = "".join(tw_df['text'])
ngrams_summaries = vect.build_analyzer()(summaries)

Counter(ngrams_summaries).most_common(20)

[('trading alerts', 11),
 ('aapl amzn', 8),
 ('aapl https', 7),
 ('exciting day', 7),
 ('day market', 7),
 ('market team_blackbox', 7),
 ('team_blackbox members', 7),
 ('members trading', 7),
 ('alerts community', 7),
 ('exciting day market', 7),
 ('day market team_blackbox', 7),
 ('market team_blackbox members', 7),
 ('team_blackbox members trading', 7),
 ('members trading alerts', 7),
 ('trading alerts community', 7),
 ('exciting day market team_blackbox', 7),
 ('day market team_blackbox members', 7),
 ('market team_blackbox members trading', 7),
 ('team_blackbox members trading alerts', 7),
 ('members trading alerts community', 7)]